# pipe

> Common operations around the core datastructures for running a sim

In [1]:
#| default_exp pipe

In [2]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export

import jax
import jax.numpy as jnp
from jaxtyping import Array, Float
from typing import Callable, Union
from functools import partial

from chewc.population import Population
from chewc.sp import SimParam
from chewc.phenotype import set_pheno
from chewc.cross import make_cross, _make_cross_geno
from chewc.trait import TraitCollection

@partial(jax.jit, static_argnames=['n_ind', 'use_pheno', 'select_top'])
def select_ind(
    pop: Population,
    n_ind: int,
    use_pheno: bool = True,
    select_top: bool = True
) -> jnp.ndarray:
    """
    (JIT-compatible) Selects indices of top/bottom individuals.

    This function is now fully JIT-compatible. Python logic has been replaced
    with `jax.lax.cond`. It no longer returns a sliced Population object, but
    instead returns the *indices* of the selected individuals. This is a more
    flexible, functional approach that allows the caller to decide how to use
the indices.

    Args:
        pop: The population to select from.
        n_ind: The number of individuals to select (static argument).
        use_pheno: If True, select on phenotype; if False, select on breeding value (bv).
                   (static argument).
        select_top: If True, selects highest values; otherwise, lowest. (static argument).

    Returns:
        A 1D array of integer indices for the selected individuals.
    """
    # jax.lax.cond is JAX's traceable equivalent of an if/else statement.
    # It requires that both branches return Pytrees with the same structure.
    selection_values = jax.lax.cond(
        use_pheno,
        lambda p: p.pheno[:, 0], # True branch: use phenotype (1st trait)
        lambda p: p.bv[:, 0],    # False branch: use breeding value (1st trait)
        pop
    )

    # We can also use lax.cond to handle top/bottom selection to avoid branching.
    # We select top values by default, and top of the negative for bottom values.
    values_to_sort = jax.lax.cond(
        select_top,
        lambda x: x,
        lambda x: -x,
        selection_values
    )

    _, indices = jax.lax.top_k(values_to_sort, k=n_ind)
    return indices

def update_pop_values(
    key: jax.random.PRNGKey,
    pop: Population,
    sp: SimParam,
    h2: Float[Array, "nTraits"]
) -> Population:
    """
    (JIT-compatible) Calculates genetic and phenotypic values for a population.

    This function was already JIT-compatible as it wraps the `set_pheno` function.
    No changes were needed. It's a key step after creating progeny.
    """
    # `set_pheno` is already designed to be JIT-compatible with `ploidy` as a static arg.
    pop_with_values = set_pheno(key, pop, sp.traits, sp.ploidy, h2=h2)
    return pop_with_values



# The NEW JIT-compiled simulation engine
@partial(jax.jit, static_argnames=[
    'n_parents', 'n_crosses', 'use_pheno_selection', 'select_top_parents', 'ploidy'
])
def run_generation(
    key: jax.random.PRNGKey,
    pop: Population,
    h2: Float[Array, "nTraits"],
    n_parents: int,
    n_crosses: int,
    # Static parameters for the JIT compilation:
    use_pheno_selection: bool,
    select_top_parents: bool,
    ploidy: int,
    # Pass static SimParam components directly
    gen_map: jnp.ndarray,
    recomb_param_v: float,
    traits: TraitCollection
) -> Population:
    """
    The complete, JIT-compiled function for a single generation step.

    This function encapsulates the entire process: selection, crossing, and
    phenotyping into a single computational graph that can be fused into a
    highly-efficient GPU/TPU kernel by XLA.

    Args:
        key: A JAX random key for this entire generation.
        pop: The starting population for the generation.
        h2: Heritability for phenotyping the next generation.
        n_parents: How many parents to select (static).
        n_crosses: How many crosses to make (static).
        use_pheno_selection: Select on phenotype (True) or BV (False) (static).
        select_top_parents: Select top (True) or bottom (False) parents (static).
        ploidy: The ploidy of the individuals (static).
        gen_map: The genetic map from SimParam.
        recomb_param_v: The recombination interference parameter from SimParam.
        traits: The TraitCollection from SimParam.

    Returns:
        The new Population object for the next generation, with all values computed.
    """
    key_select, key_cross_plan, key_cross_geno, key_pheno = jax.random.split(key, 4)

    # 1. Select Parents (JIT-compatible)
    # Corrected code in run_generation
    parent_indices = select_ind(
        pop, n_parents, use_pheno_selection, select_top_parents
    )
    parent_pool_geno = pop.geno[parent_indices]
    parent_pool_iids = pop.iid[parent_indices]

    # 2. Generate a Random Cross Plan (JIT-compatible)
    mother_iids_indices = jax.random.choice(key_cross_plan, parent_pool_iids.shape[0], shape=(n_crosses,))
    key_cross_plan, _ = jax.random.split(key_cross_plan) # Consume key
    father_iids_indices = jax.random.choice(key_cross_plan, parent_pool_iids.shape[0], shape=(n_crosses,))

    mothers_geno = parent_pool_geno[mother_iids_indices]
    fathers_geno = parent_pool_geno[father_iids_indices]
    
    # Keep track of parent public IDs for the new population's pedigree
    parent_pool_public_ids = pop.id[parent_indices]
    mother_public_ids = parent_pool_public_ids[mother_iids_indices]
    father_public_ids = parent_pool_public_ids[father_iids_indices]

    # 3. Create Progeny Genotypes (using the JIT-ted core)
    progeny_geno = _make_cross_geno(
        key_cross_geno,
        mothers_geno,
        fathers_geno,
        gen_map.shape[0], # n_chr is static from shape
        gen_map,
        recomb_param_v
    )

    # 4. Wrap results in a temporary Population object (no CPU logic here)
    # The IDs will be placeholders until the full sim loop updates them.
    progeny_pop_no_values = Population(
        geno=progeny_geno,
        id=jnp.arange(n_crosses), # Placeholder public ID
        iid=jnp.arange(n_crosses),# Internal ID
        mother=mother_public_ids,
        father=father_public_ids,
        sex=jax.random.choice(key, jnp.array([0,1], dtype=jnp.int8), (n_crosses,)),
        pheno=jnp.zeros((n_crosses, traits.n_traits)),
        fixEff=jnp.zeros(n_crosses, dtype=jnp.float32),
        bv=jnp.zeros((n_crosses, traits.n_traits)),
    )
    
    # 5. Calculate BV and Phenotypes for the new generation (JIT-compatible)
    final_progeny_pop = set_pheno(key_pheno, progeny_pop_no_values, traits, ploidy, h2=h2)

    return final_progeny_pop


In [6]:
#| hide
import nbdev; nbdev.nbdev_export()